In [1]:
import os
import h5py
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statistics import stdev, mean

from scipy import signal

from sklearn.cross_decomposition import CCA

from fooof.plts.spectra import plot_spectrum, plot_spectrum_shading

from neurodsp.filt import filter_signal
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series

## 1. Function to read .mat files

In [2]:
def read_matlab(filename):
    def conv(path=''):
        p = path or '/'
        paths[p] = ret = {}
        for k, v in f[p].items():
            if type(v).__name__ == 'Group':
                ret[k] = conv(f'{path}/{k}')  # Nested struct
                continue
            v = v[()]  # It's a Numpy array now
            if v.dtype == 'object':
                # HDF5ObjectReferences are converted into a list of actual pointers
                ret[k] = [r and paths.get(f[r].name, f[r].name) for r in v.flat]
            else:
                # Matrices and other numeric arrays
                ret[k] = v if v.ndim < 2 else v.swapaxes(-1, -2)
        return ret

    paths = {}
    with h5py.File(filename, 'r') as f:
        return conv()

## 2. Parameters that can be changed

In [3]:
# Days
days = ('/Day1', '/Day2')

# Subjects
subjects = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 
            'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 
            'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30']

In [4]:
# Parieto, parieto-occipital and occipital channels
channels = (17,18,19,20,21,22,23,24,25,26,27,29,30)
channels = (23,24,25)

In [5]:
# Sampling rate
fs = 200

# nperseg
nperseg = 500

# Initialize dataframe to store variables
df = pd.DataFrame([], index = [], columns = ['subject', 'day', 'target', \
                                             'offset', 'knee', 'exponent', \
                                             'CF', 'PW', 'BW', 'n_peaks', \
                                             'r_squared', 'error'])

# II. Canonical Correlation Analysis

In [10]:
## Read EEG, apply bandpass filter, calculate PSD (Welch), canonical correlation analysis
tic = time.time()

# Initialize
accuracy_LOW = []
accuracy_MID = []
accuracy_HIGH = []

for subject in subjects: 
    for day in days:
       
        # Change working directory
        original = os.getcwd()
        os.chdir("C:/Users/ThinkPad/Desktop/Thesis/Data/"+subject+day)

        # Giving directory name
        folderdir = 'C:/Users/ThinkPad/Desktop/Thesis/Data/'+subject+day

        # Giving file extensions
        ext = ('.mat')

        # Iterating over directory
        for path, dirc, files in os.walk(folderdir):
            for file in files:
                if file.endswith(ext) and file.startswith('cnt'):
                    print(subject, day, file)

                    # Initialize lists to store Booleans
                    correct_LOW = []
                    correct_MID = []
                    correct_HIGH = []

                    # LOW, MID, or HIGH
                    level = file[4:-7]
                    
                    # Set frequency range
                    if level=='LOW':
                        fr = (3,9)
                    elif level=='MID':
                        fr = (18,24)
                    elif level=='HIGH':
                        fr = (38,44)

                    # Session 1, or session 2
                    session = file[-6]
                    
                    # Read data
                    data = read_matlab(file)['cnt']['x']
                    data_ = data.copy()
                    
                    # Read timestamps 
                    timestamps = (read_matlab(folderdir+'/mrk_'+level+'('+session+').mat')['mrk']['time'])[0]

                    # Read stimuli order
                    order_stimuli = read_matlab(folderdir+'/mrk_'+level+'('+session+').mat')['mrk']['event']['desc']

                    for trial, timestamp in enumerate(timestamps):  
                        if trial%2 == False:
                            start = int(timestamp)
                        else:
                            stop = int(timestamp)
                                                                               
                            # Stimulus frequency
                            if level=='LOW':
                                if order_stimuli[trial-1]==[1.]: 
                                    f = 5
                                elif order_stimuli[trial-1]==[2.]:
                                    f = 5.5
                                elif order_stimuli[trial-1]==[3.]:
                                    f = 6
                                elif order_stimuli[trial-1]==[4.]:
                                    f = 6.5
                            elif level=='MID':
                                if order_stimuli[trial-1]==[1.]: 
                                    f = 21
                                elif order_stimuli[trial-1]==[2.]: 
                                    f = 21.5
                                elif order_stimuli[trial-1]==[3.]: 
                                    f = 22
                                elif order_stimuli[trial-1]==[4.]: 
                                    f = 22.5
                            elif level=='HIGH':
                                if order_stimuli[trial-1]==[1.]: 
                                    f = 40
                                elif order_stimuli[trial-1]==[2.]:
                                    f = 40.5
                                elif order_stimuli[trial-1]==[3.]: 
                                    f = 41
                                elif order_stimuli[trial-1]==[4.]: 
                                    f = 41.5
#                             if f != 21.5 and level=='MID':
                            # Initialize dataframe
                            X = pd.DataFrame([]) 

                            for channel in channels:

                                # Apply bandpass filter
                                data[:,channel] = filter_signal(data[:,channel], fs, 'bandpass', f_range=fr)

#                                 # Plot EEG 
#                                 times = create_times(len(data)/fs, fs)
#                                 fig, ax = plt.subplots(figsize=[15, 5])
#                                 plot_time_series(times, data[:,channel], xlim=[0,len(data[:,channel])/fs], ax=ax)
#                                 plt.show()    

                                # Splice data according to timestamps
                                sig = data[:,channel][start:stop]
                                times = create_times(len(sig)/fs, fs) 

                                # Welch FFT: calculate mean of spectrogram
                                freq, power = signal.welch(sig, fs, window='hann', average='mean', nperseg=nperseg,\
                                                           noverlap=nperseg*0.9, detrend='constant')

                                # Plot PSD
#                                 fig, ax = plt.subplots(figsize=[15, 4])
#                                 plot_spectrum(freq[:126], power[:126], log_powers=True, \
#                                               label=(channel, order_stimuli[trial-1], level), ax=ax)
#                                 plot_spectrum_shading(freq[:126], power[:126], [7, 12], log_powers=True, ax=ax)
#                                 plt.show()

                                X = pd.concat([X, pd.DataFrame(power)], axis=1)  

                            # Square wave for stimulus
                            t = np.linspace(0, len(sig)/fs, len(sig), endpoint=False)
                            square = signal.square(2 * np.pi * f * t)
                            
                            # Welch FFT of stimulus square wave
                            freq_Y, power_Y = signal.welch(square, fs, window='hann', average='mean', nperseg=nperseg, \
                                                         noverlap=nperseg*0.9, detrend='constant')
                            
#                             # Plot PSD
#                             fig, ax = plt.subplots(figsize=[15, 4])
#                             plot_spectrum(freq_Y, power_Y, log_powers=True, \
#                                           label=(channel, order_stimuli[trial-1], level), ax=ax)
#                             plt.show()

                            # Initialize
                            cca = CCA(n_components=1)

                            # Fit the model
                            try:
                                X_c, Y_c = cca.fit_transform(X, power_Y)
                                if level=='LOW':
                                    correct_LOW.append(freq[X_c.argmax()]-f<=0.2)
                                elif level=='MID':
                                    correct_MID.append(freq[X_c.argmax()]-f<=0.2)
                                elif level=='HIGH':
                                    correct_HIGH.append(freq[X_c.argmax()]-f<=0.2)
                            except:
                                pass

                    if level=='LOW':
                        accuracy_LOW.append(sum(correct_LOW)/len(correct_LOW))
                    elif level=='MID':
                        accuracy_MID.append(sum(correct_MID)/len(correct_MID))
                    elif level=='HIGH':
                        accuracy_HIGH.append(sum(correct_HIGH)/len(correct_HIGH))

# Change working directory back
os.chdir(original)

print(f'Done in {time.time()-tic} seconds')

S1 /Day1 cnt_HIGH(1).mat


C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transiti

S1 /Day1 cnt_HIGH(2).mat


C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transiti

S1 /Day1 cnt_LOW(1).mat
S1 /Day1 cnt_LOW(2).mat
S1 /Day1 cnt_MID(1).mat
S1 /Day1 cnt_MID(2).mat
S1 /Day2 cnt_HIGH(1).mat


...

S30 /Day1 cnt_LOW(1).mat
S30 /Day1 cnt_LOW(2).mat
S30 /Day1 cnt_MID(1).mat
S30 /Day1 cnt_MID(2).mat
S30 /Day2 cnt_HIGH(1).mat


C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transiti

S30 /Day2 cnt_HIGH(2).mat


C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transition bandwidth is  {:.1f}  Hz. This is greater than the desired'\
C:\Users\ThinkPad\anaconda3\lib\site-packages\neurodsp\filt\checks.py:172: UserWarning: Transition bandwidth is  12.0  Hz. This is greater than the desiredpass/stop bandwidth of  6.0 Hz
  warn('Transiti

S30 /Day2 cnt_LOW(1).mat
S30 /Day2 cnt_LOW(2).mat
S30 /Day2 cnt_MID(1).mat
S30 /Day2 cnt_MID(2).mat
Done in 2568.465581178665 seconds


In [11]:
pd.DataFrame([accuracy_LOW]).to_csv('accuracy_LOW_13.csv')
pd.DataFrame([accuracy_MID]).to_csv('accuracy_MID_3.csv')
pd.DataFrame([accuracy_MID]).to_csv('accuracy_3_3.csv')
pd.DataFrame([accuracy_HIGH]).to_csv('accuracy_HIGH_13.csv')